In [85]:
import pandas as pd
import numpy as np
import xarray as xr
import calliope

In [86]:
# Load your weather data
df = pd.read_excel("WeatherData.xlsx",index_col=0)

In [87]:
df.head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,Q,SQ,VV,N,U,P
0,330,20240101,1,210,120,120,180,75,NaN,61,0,0,NaN,NaN,91,9951
1,330,20240101,2,210,90,100,150,74,NaN,62,0,0,NaN,NaN,91,9958
2,330,20240101,3,240,100,110,160,92,NaN,59,0,0,NaN,NaN,79,9959
3,330,20240101,4,240,110,110,190,91,NaN,59,0,0,NaN,NaN,80,9966
4,330,20240101,5,240,100,100,150,95,NaN,54,0,0,NaN,NaN,76,9973


In [88]:
# Ensure 'HH' is properly formatted
df["HH"] = df["HH"].astype(str).str.zfill(2)

In [89]:
# Handle '24' hour values by shifting to next day
def parse_datetime(row):
    hour = int(row["HH"])
    date = pd.to_datetime(str(row["YYYYMMDD"]), format="%Y%m%d")
    if hour == 24:
        return date + pd.Timedelta(days=1)
    else:
        return date + pd.Timedelta(hours=hour)

df["datetime"] = df.apply(parse_datetime, axis=1)
df.set_index("datetime", inplace=True)

In [90]:
# New column names 
new_column_names = { 
    'FH' : 'average windspeed (m/s)',
    'FF' : 'windspeed previous 10 minuten (m/s)',
    'FX' : 'highest windspeed (m/s)',
    'T'  : 'temperature_C',
    'SQ' : 'duration of sunshine (h)',
    'Q'  : 'global_radiation_KW',    
    'P'  : 'air pressure (Pa)',
    'N'  : 'cloud cover (-)',
    'U'  : 'relative humidity (in percentages)'
}
df = df.rename(columns = new_column_names)


In [91]:
yeild_data_df=pd.read_csv('data_experimenting/df_with_calculations.csv',index_col=0)

In [92]:
yeild_data_df.head()

,YYYYMMDDHH,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature in degrees Celcius,global radiation (in J/m^2),duration of sunshine (h),cloud cover (-),relative humidity (in percentages),air pressure (Pa),yield solar panel (kWh)
0,2024-01-01 01:00:00,120,120,180,75,0.0,0.0,NaN,91,9951,0.0
1,2024-01-01 02:00:00,90,100,150,74,0.0,0.0,NaN,91,9958,0.0
2,2024-01-01 03:00:00,100,110,160,92,0.0,0.0,NaN,79,9959,0.0
3,2024-01-01 04:00:00,110,110,190,91,0.0,0.0,NaN,80,9966,0.0
4,2024-01-01 05:00:00,100,100,150,95,0.0,0.0,NaN,76,9973,0.0


In [93]:
energy_demand=yeild_data_df["yield solar panel (kWh)"]

In [94]:
energy_demand.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: yield solar panel (kWh), dtype: float64

In [95]:
df["energy_demand"]= energy_demand.values

In [96]:
df.head()

,STN,YYYYMMDD,HH,DD,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature_C,T10N,TD,global_radiation_KW,duration of sunshine (h),VV,cloud cover (-),relative humidity (in percentages),air pressure (Pa),energy_demand
datetime,,,,,,,,,,,,,,,,,
2024-01-01 01:00:00,330,20240101,01,210,120,120,180,75,NaN,61,0,0,NaN,NaN,91,9951,0.0
2024-01-01 02:00:00,330,20240101,02,210,90,100,150,74,NaN,62,0,0,NaN,NaN,91,9958,0.0
2024-01-01 03:00:00,330,20240101,03,240,100,110,160,92,NaN,59,0,0,NaN,NaN,79,9959,0.0
2024-01-01 04:00:00,330,20240101,04,240,110,110,190,91,NaN,59,0,0,NaN,NaN,80,9966,0.0
2024-01-01 05:00:00,330,20240101,05,240,100,100,150,95,NaN,54,0,0,NaN,NaN,76,9973,0.0


In [97]:
# Convert raw values
df["temperature_C"] = df["temperature_C"] / 10
df["global_radiation_KW"] = (df["global_radiation_KW"] * 1e4) / (1e3 * 3600 )
df["duration of sunshine (h)"] = df["duration of sunshine (h)"] * 0.1

In [98]:
df.sample(5)

,STN,YYYYMMDD,HH,DD,average windspeed (m/s),windspeed previous 10 minuten (m/s),highest windspeed (m/s),temperature_C,T10N,TD,global_radiation_KW,duration of sunshine (h),VV,cloud cover (-),relative humidity (in percentages),air pressure (Pa),energy_demand
datetime,,,,,,,,,,,,,,,,,
2024-01-21 09:00:00,330,20240121,09,200,110,110,160,3.0,NaN,-9,0.019444,0.0,NaN,NaN,76,10174,0.021813
2024-01-26 22:00:00,330,20240126,22,260,60,60,80,6.3,NaN,25,0.000000,0.0,NaN,NaN,76,10336,0.000000
2024-01-01 04:00:00,330,20240101,04,240,110,110,190,9.1,NaN,59,0.000000,0.0,NaN,NaN,80,9966,0.000000
2024-01-08 23:00:00,330,20240108,23,70,90,90,130,-2.5,NaN,-74,0.000000,0.0,NaN,NaN,69,10337,0.000000
2024-01-23 12:00:00,330,20240123,12,200,80,70,120,8.3,56.0,48,0.205556,0.6,NaN,NaN,78,10229,0.111417


In [99]:
# Keep only relevant columns
df_clean = df[["global_radiation_KW", "energy_demand"]]

In [100]:
df_clean.sample(5)

,global_radiation_KW,energy_demand
datetime,,
2024-01-10 14:00:00,0.188889,0.064230
2024-01-09 08:00:00,0.000000,0.000000
2024-01-21 12:00:00,0.077778,0.042404
2024-01-11 22:00:00,0.000000,0.000000
2024-01-27 00:00:00,0.000000,0.000000


In [101]:
df_clean.to_csv("solar_model/time_series/irradiance.csv")

In [102]:
# Constants
area_of_solar_panel = 2.47  # in m^2
efficiency_solar_panel = 0.17  # unitless efficiency
number_of_solar_panels = 1  # if multiple, change this

# Calculate irradiance in kWh for each hour
irradiance_kwh = (df['air pressure (Pa)'] * area_of_solar_panel * number_of_solar_panels * efficiency_solar_panel) / (1e3 * 3600)
irradiance_kwh.name = "nl_site"  # Important for Calliope

# Save to CSV
irradiance_kwh.to_csv("solar_model/time_series/irradiance.csv", header=True)


In [103]:
# Save CSV with location name as header
irradiance_kwh.to_csv("solar_model/time_series/irradiance.csv", header=["nl_site"])



In [104]:
pd.read_csv("solar_model/time_series/irradiance.csv", header=None).head()

,0,1
0,datetime,nl_site
1,2024-01-01 01:00:00,0.0011606735833333335
2,2024-01-01 02:00:00,0.0011614900555555559
3,2024-01-01 03:00:00,0.0011616066944444446
4,2024-01-01 04:00:00,0.001162423166666667


In [105]:
# Check row counts per day
daily_counts = df.groupby(df.index.date).size()
print(daily_counts.value_counts())  # Should show 24s only

24    30
23     1
1      1
dtype: int64


In [106]:
# To find problematic days
print(daily_counts[daily_counts != 24])


2024-01-01    23
2024-02-01     1
dtype: int64


In [107]:
import calliope

In [110]:
# Load and inspect the CSV
df = pd.read_csv("solar_model/time_series/irradiance.csv")

# Display first few rows and column names
print(df.columns)
print(df.head())

Index(['datetime', 'nl_site'], dtype='object')
              datetime   nl_site
0  2024-01-01 01:00:00  0.001161
1  2024-01-01 02:00:00  0.001161
2  2024-01-01 03:00:00  0.001162
3  2024-01-01 04:00:00  0.001162
4  2024-01-01 05:00:00  0.001163


In [111]:
model = calliope.Model("solar_model/model.yaml")


Parser error when reading YAML from solar_model/model.yaml.


ScannerError: mapping values are not allowed in this context
  in "<unicode string>", line 19, column 23

In [ ]:
# We can plot this by using the timeseries plotting functionality.
# The top-left dropdown gives us the chance to scroll through other timeseries data too.

model.plot.timeseries()